In [12]:
import sys
sys.path.append(sys.path[0]+'/../..')

import utils.climatology

In [13]:
import xarray
import numpy
import pandas

# Make up some test data

In [26]:
 # Create an xarray of cftime.datetime objects
testTime = xarray.cftime_range(start="1200-01-15", periods=1000, freq="M", calendar="noleap")
startYear=1208
finishYear=1225

In [27]:
testData = 1000+10*numpy.random.randn(1000,1)

testDa = xarray.DataArray(
    testData, 
    dims=('time', 'index'), 
    coords = {'time':testTime}
)

# Test date interval

In [28]:
expClimat=testData[(startYear-1200)*12:(finishYear-1200+1)*12].copy()

In [29]:
#This is a efficient but unconvincing (due to obfuscation) way of testing it

if numpy.alltrue(
    numpy.equal(expClimat, 
                utils.climatology.dateInterval(testDa, startYear, finishYear).values
               )
):
    print('Success, utils.climatology.dateInterval works')   
else:
    raise RuntimeError('Error found. testClimatology of dateInterval failed')
    

Success, utils.climatology.dateInterval works


In [30]:
# the harder way to stuff up of testing that

calculatedClimat=utils.climatology.dateInterval(testDa, startYear, finishYear)

for i in range(0,len(expClimat)):
    if calculatedClimat[i]!=expClimat[i]:
        raise RuntimeError('Error found. testClimatology of dateInterval failed')
        

# Test normalise

In [31]:
# For my climatology, calculate mean and std dev

expMean = list()
expStd = list()
expNormalised = list()

for iMonth in range(0,12):
    monthsC=numpy.arange(iMonth, 12*(finishYear-startYear+1), 12)
    monthClimat=(expClimat[monthsC])
    expMean=(numpy.mean(monthClimat))
    expStd=(numpy.std(monthClimat))
    monthsT=numpy.arange(iMonth, len(testData), 12)
    expNormalised.append([(testData[monthsT]-expMean)/expStd])


In [32]:
calculatedNormalised=utils.climatology.normalise(testDa, calculatedClimat)

In [33]:
for iMonth in range(0,12):
    calcNorm=calculatedNormalised.where(calculatedNormalised.month==iMonth+1, drop=True).values
    for i in range(0,len(expNormalised)):
        if not numpy.isclose(calcNorm[i],expNormalised[iMonth][0][i]):
            raise RuntimeError('Error found. testClimatology of dateInterval failed')
    